## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kandry,RU,54.5667,54.1167,-3.77,89,100,15.86,overcast clouds
1,1,Kormilovka,RU,55.0026,74.1028,28.40,93,75,15.66,light snow
2,2,Qaanaaq,GL,77.4840,-69.3632,9.77,86,99,6.98,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,53.60,71,40,19.57,scattered clouds
4,4,Przasnysz,PL,53.0191,20.8803,10.00,84,0,7.25,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Rikitea,PF,-23.1203,-134.9692,78.44,69,100,9.95,overcast clouds
12,12,Mataura,NZ,-46.1927,168.8643,73.74,53,0,2.39,clear sky
13,13,Kahului,US,20.8947,-156.4700,77.00,50,20,11.50,few clouds
17,17,Rosarito,MX,32.3333,-117.0333,77.00,48,1,6.91,clear sky
26,26,Cape Town,ZA,-33.9258,18.4232,71.01,83,20,9.22,few clouds
32,32,Sao Joao Da Barra,BR,-21.6403,-41.0511,72.23,95,100,13.15,heavy intensity rain
34,34,Sao Filipe,CV,14.8961,-24.4956,70.09,73,42,14.50,scattered clouds
40,40,Mount Isa,AU,-20.7333,139.5000,75.20,100,90,8.05,light rain
45,45,Saint-Pierre,RE,-21.3393,55.4781,78.80,83,40,4.61,light rain
57,57,Arraial Do Cabo,BR,-22.9661,-42.0278,75.63,83,100,5.86,moderate rain


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().values.any()

True

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_city_df=preferred_cities_df.dropna()

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Rikitea,PF,78.44,overcast clouds,-23.1203,-134.9692,
12,Mataura,NZ,73.74,clear sky,-46.1927,168.8643,
13,Kahului,US,77.00,few clouds,20.8947,-156.4700,
17,Rosarito,MX,77.00,clear sky,32.3333,-117.0333,
26,Cape Town,ZA,71.01,few clouds,-33.9258,18.4232,
32,Sao Joao Da Barra,BR,72.23,heavy intensity rain,-21.6403,-41.0511,
34,Sao Filipe,CV,70.09,scattered clouds,14.8961,-24.4956,
40,Mount Isa,AU,75.20,light rain,-20.7333,139.5000,
45,Saint-Pierre,RE,78.80,light rain,-21.3393,55.4781,
57,Arraial Do Cabo,BR,75.63,moderate rain,-22.9661,-42.0278,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.drop(hotel_df[hotel_df['Hotel Name']==''].index, inplace = True)

In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))